In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.v2 as transforms_v2
import sys
import os
import time
import random
import numpy as np
import wandb
from tqdm import tqdm
from tools.tool import AccuracyEarlyStopping, WarmUpLR, SAM
# Wide ResNet 모델 임포트
from models.wideresidual import Wide_ResNet, conv_init

# 중요: wideresidual.py 파일에서 "*wide*layer" 메서드명을 "_wide_layer"로 수정해야 합니다!

# 체크포인트 저장 및 불러오기 함수 정의
def save_checkpoint(model, optimizer, main_scheduler, epoch, best_acc, save_path):
    """모델 체크포인트를 저장합니다."""
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.module.state_dict() if isinstance(model, nn.DataParallel) else model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': main_scheduler.state_dict() if main_scheduler else None,
        'best_acc': best_acc
    }
    torch.save(checkpoint, save_path)
    print(f"체크포인트가 {save_path}에 저장되었습니다.")

def load_checkpoint(model, optimizer, main_scheduler, checkpoint_path):
    """저장된 체크포인트를 불러옵니다."""
    if not os.path.exists(checkpoint_path):
        print(f"체크포인트 파일이 존재하지 않습니다: {checkpoint_path}")
        return model, optimizer, main_scheduler, 0, 0
    
    checkpoint = torch.load(checkpoint_path)
    start_epoch = checkpoint['epoch'] + 1
    
    # 모델 가중치 불러오기
    if isinstance(model, nn.DataParallel):
        model.module.load_state_dict(checkpoint['model_state_dict'])
    else:
        model.load_state_dict(checkpoint['model_state_dict'])
    
    # 옵티마이저 상태 불러오기
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    # 스케줄러 상태 불러오기 (있는 경우)
    if main_scheduler and checkpoint['scheduler_state_dict']:
        main_scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    
    print(f"{checkpoint_path}에서 에폭 {start_epoch}의 체크포인트를 불러왔습니다.")
    return model, optimizer, main_scheduler, start_epoch, checkpoint['best_acc']

# 학습 재개 설정 (필요에 따라 변경)
resume_training = False  # True로 설정하면 체크포인트에서 학습 재개
checkpoint_path = 'checkpoint.pt'  # 체크포인트 파일 경로

# WandB 설정
wandb.login(key="ef091b9abcea3186341ddf8995d62bde62d7469e")

# 기존 실행을 계속할지 새 실행을 시작할지 설정
if resume_training and os.path.exists(checkpoint_path):
    # 기존 실행 ID를 사용하려면 아래 주석을 해제하고 ID를 입력하세요
    # wandb.init(project="PBL-2", name="wideresnet_28_20_cfc,SGD", id="기존_실행_ID", resume="must")
    # 또는 새 실행으로 시작하되 동일한 이름 사용
    wandb.init(project="PBL-2", name="wideresnet_28_20_cfc,SGD")
else:
    # 새 실행 시작
    wandb.init(project="PBL-2", name="wideresnet_28_20_cfc,SGD")

# WandB 설정 - WideResNet 28x20으로 변경
config = {
    "model": "wideresnet",  
    "depth": 28,            
    "widen_factor": 20,     # 10에서 20으로 변경
    "dropout_rate": 0.3,    
    "batch_size": 128,       # 메모리 부족 가능성 고려하여 더 작은 배치 크기로 조정
    "num_epochs": 300,
    "learning_rate": 0.01,  
    "optimizer": "SGD",     
    "momentum": 0.9,        
    "weight_decay": 5e-4,   
    "deterministic": False,
    "patience": 30,         
    "max_epochs_wait": float('inf'),
    "cutmix_alpha": 1.0,    
    "cutmix_prob": 0.5,     
    "crop_padding": 4,      
    "crop_size": 32,        
    "warmup_epochs": 5,     
}
wandb.config.update(config)


# CIFAR-100 데이터셋 로드 - 강화된 데이터 증강 추가
transform_train = transforms.Compose([
    transforms.RandomCrop(config["crop_size"], padding=config["crop_padding"]),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

trainset = torchvision.datasets.CIFAR100(
    root='./data', train=True, download=True, transform=transform_train)

testset = torchvision.datasets.CIFAR100(
    root='./data', train=False, download=True, transform=transform_test)

# DataLoader 생성
trainloader = DataLoader(trainset, batch_size=config["batch_size"], shuffle=True, 
                        num_workers=32, pin_memory=True)  # GPU 전송 속도 향상
testloader = DataLoader(testset, batch_size=config["batch_size"], shuffle=False, 
                       num_workers=32, pin_memory=True)

print(f"Train set size: {len(trainset)}")
print(f"Test set size: {len(testset)}")

# CutMix 변환 정의
cutmix = transforms_v2.CutMix(alpha=config["cutmix_alpha"], num_classes=100)  # CIFAR-100은 100개 클래스

def train(model, trainloader, criterion, optimizer, device, epoch, warmup_scheduler=None, warmup_epochs=None):
    """
    학습 함수 (CutMix 적용) - SGD 최적화용으로 수정
    """
    if warmup_epochs is None:
        warmup_epochs = config["warmup_epochs"]
        
    model.train()   # 모델을 학습 모드로 설정
    start_time = time.time()  # 시간 측정 시작
    running_loss = 0.0
    correct_top1 = 0
    correct_top5 = 0
    total = 0
    
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        # CutMix 확률적 적용
        if random.random() < config["cutmix_prob"]:
            inputs, labels = cutmix(inputs, labels)
            use_cutmix = True
        else:
            use_cutmix = False
        
        # 그래디언트 초기화
        optimizer.zero_grad()
        
        # 순전파
        outputs = model(inputs)
        
        # CutMix 적용 여부에 따라 손실 함수 선택
        if use_cutmix:
            loss = torch.nn.functional.cross_entropy(outputs, labels)
        else:
            loss = criterion(outputs, labels)
            
        # 역전파
        loss.backward()
        
        # 가중치 업데이트 (단일 스텝)
        optimizer.step()
        
        # 학습률 스케줄러 업데이트
        if epoch < warmup_epochs and warmup_scheduler is not None:
            warmup_scheduler.step()
        
        running_loss += loss.item()

        # 정확도 계산
        if use_cutmix:
            _, label_idx = labels.max(1)
        else:
            label_idx = labels
        
        # top-1 정확도 계산
        _, predicted = outputs.max(1)
        total += inputs.size(0)
        correct_top1 += predicted.eq(label_idx).sum().item()
        
        # top-5 정확도 계산
        _, top5_idx = outputs.topk(5, 1, largest=True, sorted=True)
        correct_top5 += sum([1 for i in range(len(label_idx)) if label_idx[i] in top5_idx[i]])
        
        if (i + 1) % 50 == 0:  # 50 배치마다 출력
            print(f'Epoch [{epoch+1}], Batch [{i+1}/{len(trainloader)}], Loss: {loss.item():.4f}, LR: {optimizer.param_groups[0]["lr"]:.6f}')
    
    epoch_loss = running_loss / len(trainloader)
    accuracy_top1 = 100.0 * correct_top1 / total
    accuracy_top5 = 100.0 * correct_top5 / total
    
    train_time = time.time() - start_time
    
    # 학습 세트에 대한 성능 출력
    print(f'Train set: Epoch: {epoch+1}, Average loss:{epoch_loss:.4f}, LR: {optimizer.param_groups[0]["lr"]:.6f} '
          f'Top-1 Accuracy: {accuracy_top1:.4f}%, Top-5 Accuracy: {accuracy_top5:.4f}%, Time consumed:{train_time:.2f}s')
    
    return epoch_loss, accuracy_top1, accuracy_top5

def evaluate(model, dataloader, criterion, device, epoch, phase="test"):
    """
    평가 함수
    """
    model.eval()  # 모델을 평가 모드로 설정
    start_time = time.time()  # 시간 측정 시작
    
    eval_loss = 0.0
    correct_top1 = 0
    correct_top5 = 0
    total = 0
    
    # 그래디언트 계산 비활성화
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # 순전파
            outputs = model(inputs)
            
            # 손실 계산
            loss = criterion(outputs, labels)
            eval_loss += loss.item()
            
            # top-1 정확도 계산
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct_top1 += (predicted == labels).sum().item()
            
            # top-5 정확도 계산
            _, top5_idx = outputs.topk(5, 1, largest=True, sorted=True)
            correct_top5 += top5_idx.eq(labels.view(-1, 1).expand_as(top5_idx)).sum().item()
    
    # 평균 손실 및 정확도 계산
    eval_loss = eval_loss / len(dataloader)
    accuracy_top1 = 100.0 * correct_top1 / total
    accuracy_top5 = 100.0 * correct_top5 / total
    
    # 평가 시간 계산
    eval_time = time.time() - start_time
    
    # 테스트 세트에 대한 성능 출력
    print(f'{phase.capitalize()} set: Epoch: {epoch+1}, Average loss:{eval_loss:.4f}, '
          f'Top-1 Accuracy: {accuracy_top1:.4f}%, Top-5 Accuracy: {accuracy_top5:.4f}%, Time consumed:{eval_time:.2f}s')
    print()
    
    return eval_loss, accuracy_top1, accuracy_top5

def main_training_loop(model, trainloader, testloader, criterion, optimizer, device, 
                      num_epochs, patience, max_epochs_wait, warmup_scheduler=None, 
                      main_scheduler=None, warmup_epochs=None, resume=False, checkpoint_path='checkpoint.pt'):
    """
    메인 학습 루프 (학습 재개 기능 추가)
    """
    if warmup_epochs is None:
        warmup_epochs = config["warmup_epochs"]
    
    start_epoch = 0
    best_test_acc_top1 = 0.0
    best_test_acc_top5 = 0.0
    
    # 학습 재개 시 체크포인트 불러오기
    if resume and os.path.exists(checkpoint_path):
        model, optimizer, main_scheduler, start_epoch, best_test_acc_top1 = load_checkpoint(
            model, optimizer, main_scheduler, checkpoint_path
        )
        print(f"에폭 {start_epoch}부터 학습을 재개합니다. 최고 정확도: {best_test_acc_top1:.2f}%")
    
    # 정확도 기반 얼리 스토핑 사용
    early_stopping = AccuracyEarlyStopping(patience=patience, verbose=True, path='best_checkpoint.pt', max_epochs=max_epochs_wait)
    if resume and best_test_acc_top1 > 0:
        early_stopping.best_score = best_test_acc_top1 / 100.0  # AccuracyEarlyStopping에서 사용하는 스케일로 변환
    
    # 테스트 정확도 기록을 위한 리스트
    test_acc_top1_history = []
    
    # tqdm을 사용한 진행 상황 표시 (시작 에폭부터)
    for epoch in tqdm(range(start_epoch, num_epochs)):
        # 학습
        train_loss, train_acc_top1, train_acc_top5 = train(
            model, trainloader, criterion, optimizer, device, epoch, 
            warmup_scheduler, warmup_epochs
        )
        
        # 테스트 데이터로 평가
        test_loss, test_acc_top1, test_acc_top5 = evaluate(model, testloader, criterion, device, epoch, phase="test")

        # 웜업 이후 ReduceLROnPlateau 스케줄러 업데이트 
        if epoch >= warmup_epochs and main_scheduler is not None:
            main_scheduler.step(test_acc_top1)  # 테스트 정확도에 따라 학습률 업데이트
        
        # 테스트 정확도 기록
        test_acc_top1_history.append(test_acc_top1)
        
        # WandB에 로깅
        wandb.log({
            "epoch": epoch + 1,
            "learning_rate": optimizer.param_groups[0]['lr'],
            "train_loss": train_loss,
            "train_accuracy_top1": train_acc_top1,
            "train_accuracy_top5": train_acc_top5,
            "test_loss": test_loss,
            "test_accuracy_top1": test_acc_top1,
            "test_accuracy_top5": test_acc_top5
        })
        
        # 최고 정확도 모델 저장 (top-1 기준)
        if test_acc_top1 > best_test_acc_top1:
            best_test_acc_top1 = test_acc_top1
            best_test_acc_top5_at_best_top1 = test_acc_top5
            print(f'새로운 최고 top-1 정확도: {best_test_acc_top1:.2f}%, top-5 정확도: {best_test_acc_top5_at_best_top1:.2f}%')
            
            # 모델 저장
            model_path = f'best_model_{wandb.run.name}.pth'
            if isinstance(model, nn.DataParallel):
                torch.save(model.module.state_dict(), model_path)  # DataParallel 래퍼 제거
            else:
                torch.save(model.state_dict(), model_path)
            
            # WandB에 모델 아티팩트 저장
            wandb.save(model_path)
        
        # top-5 accuracy 기록 업데이트
        if test_acc_top5 > best_test_acc_top5:
            best_test_acc_top5 = test_acc_top5
            print(f'새로운 최고 top-5 정확도: {best_test_acc_top5:.2f}%')

        # 정기적으로 체크포인트 저장 (예: 10 에폭마다)
        if (epoch + 1) % 10 == 0:
            save_checkpoint(model, optimizer, main_scheduler, epoch, best_test_acc_top1, checkpoint_path)
        
        # Early stopping 체크 (test_acc_top1 기준)
        early_stopping(test_acc_top1, model, epoch)
        if early_stopping.early_stop:
            print(f"에폭 {epoch+1}에서 학습 조기 종료. 최고 성능 에폭: {early_stopping.best_epoch+1}")
            break
    
    # 학습 완료 후 최종 체크포인트 저장
    save_checkpoint(model, optimizer, main_scheduler, epoch, best_test_acc_top1, 'final_checkpoint.pt')
    
    # 훈련 완료 후 최고 모델 로드
    print("테스트 정확도 기준 최고 모델 로드 중...")
    model_path = f'best_model_{wandb.run.name}.pth'
    
    # 올바른 모델 로드를 위한 처리
    if isinstance(model, nn.DataParallel):
        model.module.load_state_dict(torch.load(model_path))
    else:
        model.load_state_dict(torch.load(model_path))

    # 최종 테스트 세트 평가
    final_test_loss, final_test_acc_top1, final_test_acc_top5 = evaluate(model, testloader, criterion, device, num_epochs-1, phase="test")
    
    print(f'완료! 최고 테스트 top-1 정확도: {best_test_acc_top1:.2f}%, 최고 테스트 top-5 정확도: {best_test_acc_top5:.2f}%')
    print(f'최종 테스트 top-1 정확도: {final_test_acc_top1:.2f}%, 최종 테스트 top-5 정확도: {final_test_acc_top5:.2f}%')
    
    # WandB에 최종 결과 기록
    wandb.run.summary["best_test_accuracy_top1"] = best_test_acc_top1
    wandb.run.summary["best_test_accuracy_top5"] = best_test_acc_top5
    wandb.run.summary["final_test_accuracy_top1"] = final_test_acc_top1
    wandb.run.summary["final_test_accuracy_top5"] = final_test_acc_top5

    # Early stopping 정보 저장
    if early_stopping.early_stop:
        wandb.run.summary["early_stopped"] = True
        wandb.run.summary["early_stopped_epoch"] = epoch+1
        wandb.run.summary["best_epoch"] = early_stopping.best_epoch+1
    else:
        wandb.run.summary["early_stopped"] = False


# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 메모리 사용량 출력 함수 (디버깅용)
def print_gpu_memory():
    if torch.cuda.is_available():
        for i in range(torch.cuda.device_count()):
            print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
            print(f"Memory Allocated: {torch.cuda.memory_allocated(i)/1024**3:.2f} GB")
            print(f"Memory Reserved: {torch.cuda.memory_reserved(i)/1024**3:.2f} GB")
            print(f"Max Memory Allocated: {torch.cuda.max_memory_allocated(i)/1024**3:.2f} GB")

# GPU 메모리 초기 상태 출력
print_gpu_memory()

# Wide ResNet 모델 초기화
model = Wide_ResNet(
    depth=config["depth"],
    widen_factor=config["widen_factor"],
    dropout_rate=config["dropout_rate"],
    num_classes=100  # CIFAR-100은 100개 클래스
).to(device)

# 가중치 초기화 적용
model.apply(conv_init)

# 모델 파라미터 수 계산 및 출력
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

num_params = count_parameters(model)
print(f'Wide-ResNet-{config["depth"]}-{config["widen_factor"]} initialized with {num_params:,} parameters')
wandb.run.summary["model_parameters"] = num_params

# 손실 함수 설정 (라벨 스무딩 사용)
criterion = nn.CrossEntropyLoss()

# SGD 옵티마이저 사용
optimizer = optim.SGD(
    model.parameters(), 
    lr=config["learning_rate"],
    momentum=config["momentum"],
    weight_decay=config["weight_decay"],
    nesterov=True  # Nesterov 모멘텀 사용
)

# WarmUpLR 스케줄러 초기화
warmup_steps = config["warmup_epochs"] * len(trainloader)
warmup_scheduler = WarmUpLR(optimizer, total_iters=warmup_steps)

# 웜업 이후 사용할 스케줄러 설정 
main_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode='max',          
    factor=0.5,          
    patience=5,          
    verbose=True,        
    threshold=0.01,      
    min_lr=1e-6
)

# WandB에 모델 구조 기록
wandb.watch(model, log="all")

# GPU 가속
if torch.cuda.device_count() > 1:
    print(f"{torch.cuda.device_count()}개의 GPU를 사용합니다.")
    model = nn.DataParallel(model)
    
# 모델 상태 요약 출력
print(f"Model: WideResNet-{config['depth']}-{config['widen_factor']}")
print(f"Batch size: {config['batch_size']}")
print(f"Learning rate: {config['learning_rate']}")
print(f"Optimizer: {config['optimizer']} with SAM")
print(f"Dropout rate: {config['dropout_rate']}")

# 학습 재개 상태 출력
if resume_training:
    print(f"학습을 재개합니다. 체크포인트 파일: {checkpoint_path}")
else:
    print("처음부터 학습을 시작합니다.")

# 훈련 시작 시간 기록
start_time = time.time()

try:
    # 메인 학습 루프 호출
    main_training_loop(
        model=model,
        trainloader=trainloader,
        testloader=testloader,
        criterion=criterion,
        optimizer=optimizer,
        device=device,
        num_epochs=config["num_epochs"],
        patience=config["patience"],
        max_epochs_wait=config["max_epochs_wait"],
        warmup_scheduler=warmup_scheduler,
        main_scheduler=main_scheduler,
        warmup_epochs=config["warmup_epochs"],
        resume=resume_training,  # 학습 재개 여부
        checkpoint_path=checkpoint_path  # 체크포인트 파일 경로
    )
except Exception as e:
    print(f"학습 중 오류 발생: {e}")
    # 오류 발생시 메모리 상태 출력
    print_gpu_memory()
    # 오류가 발생해도 체크포인트 저장을 시도
    try:
        save_checkpoint(model, optimizer, main_scheduler, epoch, best_test_acc_top1, 'error_checkpoint.pt')
        print("오류 발생 시점의 체크포인트가 저장되었습니다.")
    except Exception as checkpoint_error:
        print(f"체크포인트 저장 실패: {checkpoint_error}")
    raise

# 훈련 종료 시간 기록 및 출력
end_time = time.time()
total_time = end_time - start_time
wandb.log({"total_training_time": total_time})

print(f"전체 학습 시간: {total_time:.2f} 초")
print(f"전체 학습 시간: {total_time/3600:.2f} 시간")

# 최종 메모리 상태 출력
print_gpu_memory()

# WandB 실행 종료
wandb.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/guswls/.netrc
wandb: Currently logged in as: sokjh1310 (sokjh1310-hanyang-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Files already downloaded and verified
Files already downloaded and verified
Train set size: 50000
Test set size: 10000
Using device: cuda
GPU 0: NVIDIA RTX A5000
Memory Allocated: 0.00 GB
Memory Reserved: 0.00 GB
Max Memory Allocated: 0.00 GB
GPU 1: NVIDIA RTX A5000
Memory Allocated: 0.00 GB
Memory Reserved: 0.00 GB
Max Memory Allocated: 0.00 GB
| Wide-Resnet 28x20


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Wide-ResNet-28-20 initialized with 145,950,980 parameters
2개의 GPU를 사용합니다.
Model: WideResNet-28-20
Batch size: 128
Learning rate: 0.01
Optimizer: SGD with SAM
Dropout rate: 0.3
처음부터 학습을 시작합니다.


  0%|                                                                                                       | 0/300 [00:00<?, ?it/s]

...

Test set: Epoch: 63, Average loss:1.4959, Top-1 Accuracy: 69.2900%, Top-5 Accuracy: 90.5400%, Time consumed:14.11s

EarlyStopping 카운터: 6 / 30
Epoch [64], Batch [50/391], Loss: 2.7220, LR: 0.000625
Epoch [64], Batch [100/391], Loss: 1.3385, LR: 0.000625
Epoch [64], Batch [150/391], Loss: 0.1227, LR: 0.000625
Epoch [64], Batch [200/391], Loss: 0.1265, LR: 0.000625
Epoch [64], Batch [250/391], Loss: 0.1142, LR: 0.000625
Epoch [64], Batch [300/391], Loss: 0.0737, LR: 0.000625
Epoch [64], Batch [350/391], Loss: 1.2699, LR: 0.000625
Train set: Epoch: 64, Average loss:0.8590, LR: 0.000625 Top-1 Accuracy: 86.9080%, Top-5 Accuracy: 96.6140%, Time consumed:179.46s
Test set: Epoch: 64, Average loss:1.4652, Top-1 Accuracy: 69.6200%, Top-5 Accuracy: 90.7000%, Time consumed:13.86s

새로운 최고 top-1 정확도: 69.62%, top-5 정확도: 90.70%
새로운 최고 top-5 정확도: 90.70%
Accuracy improved (69.40% --> 69.62%). Saving model ...


 21%|██████████████████▊                                                                     | 64/300 [3:28:52<12:47:15, 195.06s/it]

Epoch [65], Batch [50/391], Loss: 0.1590, LR: 0.000625
Epoch [65], Batch [100/391], Loss: 0.1019, LR: 0.000625
Epoch [65], Batch [150/391], Loss: 2.0758, LR: 0.000625
Epoch [65], Batch [200/391], Loss: 2.3894, LR: 0.000625
Epoch [65], Batch [250/391], Loss: 0.2212, LR: 0.000625
Epoch [65], Batch [300/391], Loss: 2.3896, LR: 0.000625
Epoch [65], Batch [350/391], Loss: 2.5661, LR: 0.000625
Train set: Epoch: 65, Average loss:1.0313, LR: 0.000625 Top-1 Accuracy: 82.7560%, Top-5 Accuracy: 95.2840%, Time consumed:179.55s
Test set: Epoch: 65, Average loss:1.4091, Top-1 Accuracy: 69.7400%, Top-5 Accuracy: 90.3200%, Time consumed:13.93s

새로운 최고 top-1 정확도: 69.74%, top-5 정확도: 90.32%
Accuracy improved (69.62% --> 69.74%). Saving model ...


 22%|███████████████████                                                                     | 65/300 [3:32:09<12:45:36, 195.47s/it]

Epoch [66], Batch [50/391], Loss: 1.8073, LR: 0.000625
Epoch [66], Batch [100/391], Loss: 0.1811, LR: 0.000625
Epoch [66], Batch [150/391], Loss: 1.0971, LR: 0.000625
Epoch [66], Batch [200/391], Loss: 0.0587, LR: 0.000625
Epoch [66], Batch [250/391], Loss: 2.2613, LR: 0.000625
Epoch [66], Batch [300/391], Loss: 0.0782, LR: 0.000625
Epoch [66], Batch [350/391], Loss: 0.1281, LR: 0.000625
